In [28]:
// Breakup sources into multiple files
var fs = require('fs');
var _init = fs.readFileSync('init.js').toString();
eval(_init);
setupD3();

function isEmpty(arrOrObj) {
	if(!arrOrObj) return true;
	if(isArray(arrOrObj) && arrOrObj.length === 0) return true;
	if(typeof arrOrObj === 'object' && 
			Object.keys(arrOrObj).length === 0 ) return true;
	return false;
}

D3 - loader


In [36]:
function readAll(dir) {
    let list =fs.readdirSync(dir).filter(n => n.match(/\.js\.bak$/)).filter(name => name[0] === '_');
    let files = list.map(name => name.replace(/\.js.bak$/,'').substr(1));
    let usableFiles = files.filter(name =>!fs.existsSync(`$dir/name`));
    let existing = files.filter(name =>fs.existsSync(`$dir/name`));
    if(existing && existing.length > 0) console.log("The following directories alreaddy exist", existing)
    return files.map(file => ({dir, file, getData: getData}));
}

function getData() {
    let {dir, file} = this;
    let str = fs.readFileSync(`${dir}/_${file}.js.bak`, 'utf8').replace(/\r/g,'');
    
    let [first, ...blocks] = str.split(/\/\/@@?/);
    this.first = first;
    this.blocks = blocks;
    this.getImports = _getImports;
    return this;
}

function _getImports() {
    let {dir, file, first, blocks} = this;
    this.imports = first.split("\n").filter(line => line.startsWith('import ') );
    this.getExports = _getExports;
    return this;
}

function _getExports() {
    let {blocks} = this;
    //console.log(blocks.length);
    this.exports = blocks.map(_getExportsOfBlock);
    return this;
}

function _getExportsOfBlock(block) {
    let lines = block.split("\n");
    let exps = lines.filter(l => l.match(/export/));
    let exports = exps.map(l => l.split(/\s+/));
    let defaults = exports.filter(l => l[1]=== 'default').map(([_,__,type,name])=> ({type,name: strip(name)}));
    exports = exports.filter(l => l[1] !== 'default').map(([_,type,name])=> ({type,name: strip(name)}));
    
    if(defaults.length > 1) throw new Error("too name defaaults"+ defaults.toString());
    return {block, exports, defaults};
}

function strip(name) {
    let ix = name.indexOf('(');
    if(ix >=0 ) return name.substr(0,ix);
    return name;
}

function createDir(path, refData) {
    if(fs.existsSync(path)) throw new Error("dir "+path+" exists");
    console.log(`fs.mkdirSync(${path});`); 
    fs.mkdirSync(path);
}

function getName(dir,anExp) {
    let {defaults, exports} = anExp;
    let name = '';
    let isDefault = !isEmpty(defaults)
    if( isDefault ) name = defaults[0].name;
    else name = exports[0].name;
    let path = dir+"/"+name+'.js';
    
    return [path, isDefault, name, exports.map(e => e.name)];
}

function isFile(path) {
    return fs.existsSync(path+'.js') || fs.existsSync(path)
}
function transform(dir,inports) {
    let [parts,file] = inports.replace(/import(.*)from\s['"]([a-zA-z0-9* ._/-]+)(['"].*)/,"$1\t$2")
                    .split('\t');
    let prefix = ['.', '../'].filter(pre => isFile(dir+'/'+pre+file));
    if(prefix.length === 0) return [file,file];
    return [file, prefix[0]+file];
}   

function writeBlock(dir, aB, fileData ) {
    let [path,dflt, funcName, allExports] = fileData;
    console.log("Cretae file", path);
    let lines = aB.block.split(/\r?\n/)
        .map(l => l.replace(/\/\/##\s*/g, '')
                   .replace(/^@\s+\n/,'') );
    let imports = lines.filter(l => l.match(/import\s/));
        

    let pth = allButLast(path)
    let newImports = imports.map(l => {
        let [old,nw] = transform(pth, l );
        //console.log({pth, old,nw})
        return [l,l.replace(old,nw)]
    }).reduce((o,[k,v]) => (o[k]=v,o),{})
    newImports['@'] = '//'+path
    let data = lines.map(mapper(newImports)).join('\n');
    data = data
        .replace(/export\s+default\s+const\s+[a-zA-Z0-9_$]+\s+=/g, "export default ")
        //.replace(/export\s+const\s+[a-zA-Z0-9_$]+\s+=/g, "export default ")
    
    console.log(data);
    //console.log(block);
    fs.writeFileSync(path, data);
    return fileData;
}

function mapper(dict) {
    return l => dict[l] || l;
}

function allButLast(path) {
    let p = path.split('/');
    if(p.length > 1) return p.slice(0, p.length-1).join('/');
    return path;
}

function processFile(aFile,toDelDir) {
   let fdat =  aFile.getData().getImports().getExports();
    console.log(Object.keys(fdat));
    console.log(fdat.file, 'dir',fdat.dir);
    let dir = fdat.dir+'/'+fdat.file;
    if(toDelDir) rmdir(dir);
    createDir(dir);
    let funcs = fdat.exports.map(e => writeBlock(dir, e,getName(dir,e)) );
    console.log(writeIndex(dir, funcs));
}

function writeIndex(dir, funcs) {
    let src = "";
    let other = 0;
    let path = funcs.length >0?funcs[0][0]:undefined;
    for(let i=0; i<funcs.length; i++) {
        let [d, isDefault, func, exports] = funcs[i];
        if(isDefault) {
          src += "import _"+func+" from './"+func+"';\n";
          src += "export const "+func+" = _"+func+";\n\n"
        } else src += "export * from './"+func+"';\n";
    }

    if(path) {
        let index = allButLast(path)+"/index.js";
        fs.writeFileSync(index, src);
        return {index, src};
    }
    return undefined;
}


/*
    let block = aB.block.replace(/\/\/##/g, '').replace(/^@\n/,'')
                .replace(/"/g, "'")
                .replace(/'\.\/haveFrame/, "'../haveFrame")                
                .replace(/\.\/frame/, '../frame')
                .replace(/\.\.\/string\//g, '../../string/')
                .replace(/\.\.\/utils\//, '../../utils/')
                .replace(/\.\.\/array\//, '../../array /')
                ;
*/
function rmdir(path) {
    try {
        path = path || './';
        let files = fs.readdirSync(path);
        files.forEach( f => (console.log(f), fs.unlinkSync(`${path}/${f}`)));
    } catch(e) {
        console.log(e);
    }
    try {
        console.log("rmdir ", path);
        fs.rmdirSync(path)
    }catch(e) {
        console.log(e);
    }
}
function processDir(dir) {
    var files = readAll(dir);
    console.log(files);
    files.forEach(f => processFile(f,true));  
}

undefined

In [41]:

processDir("../src/frame");
processDir("../src/array");
processDir("../src/string");
processDir("../src/dict");
processDir("../src/utils");

// var files = readAll();
// rmdir("../src/frame/frame-utils")
// //fs.rmdirSync("../src/frame/frame-utils")
// console.log(files.map(e => e.file))
// //console.log(files[0].getData().getImports().getExports().exports.map(({defaults},i) =>( defaults)));
// files.forEach(f => processFile(f,true));


[
  {
    dir: '../src/frame',
    file: 'frame-utils',
    getData: [Function: getData]
  },
  {
    dir: '../src/frame',
    file: 'join-utils',
    getData: [Function: getData]
  }
]
[
  'dir',     'file',
  'getData', 'first',
  'blocks',  'getImports',
  'imports', 'getExports',
  'exports'
]
frame-utils dir ../src/frame
frameCount.js
frameFromBuffer.js
frameSum.js
frameToString.js
frameWithIndex.js
index.js
transpose.js
unionFrame.js
rmdir  ../src/frame/frame-utils
fs.mkdirSync(../src/frame/frame-utils);
Cretae file ../src/frame/frame-utils/frameFromBuffer.js

import {Frame} from '../frame';
import {dataSplit} from '../../string/csv';
/**
 * default splitter is pipe seperated values
 * @param  {[type]} buffer    [description]
 * @param  {[type]} frameName [description]
 * @param  {[type]} splitter  [description]
 * @return {[type]}           [description]
 */
export default function frameFromBuffer(buffer,frameName,splitter) {
	let arr = buffer.split('\n');
	let columns = splitte

[
  'dir',     'file',
  'getData', 'first',
  'blocks',  'getImports',
  'imports', 'getExports',
  'exports'
]
join-utils dir ../src/frame
changeNameTo.js
colName.js
EMPTY_KEY.js
fullInnerJoin.js
getDict.js
getTransform.js
index.js
indexFrameColumn.js
innerJoin.js
joinOp.js
leftJoin.js
localGroupBy.js
outerJoin.js
project.js
rmdir  ../src/frame/join-utils
fs.mkdirSync(../src/frame/join-utils);
Cretae file ../src/frame/join-utils/EMPTY_KEY.js

export default  '@#$-EMPTY-$#@';


Cretae file ../src/frame/join-utils/changeNameTo.js

/**
 * changeNameTo takes a change name description array of two elements containing the old names and the new names
 * e.g oldName=newName 'a name change descriptor' where '=' is the seperator
 * return an array of 2 strings [oldName, newName]
 * @param  {string} name old name to new name descriptor, 'OLD=NEW'
 * @param  {string} sep  string that seperates the old name from the new
 * @return {[oldName, newName]}      and ar
 */
export default function chang

Cretae file ../src/frame/join-utils/getDict.js

import EMPTY_KEY from './EMPTY_KEY';
import {EMPTY_ARRAY} from '../../utils/constants';
export default function getDict(dict, key, noEmpty) { /*local*/
	if(noEmpty && !key) return EMPTY_ARRAY;
	key = key || EMPTY_KEY;
	return dict[key] || EMPTY_ARRAY;
}


Cretae file ../src/frame/join-utils/innerJoin.js

import haveFrame from '../haveFrame';
import colName from './colName';
import localGroupBy from './localGroupBy';
import changeNameTo from './changeNameTo';
import joinOp from './joinOp';
/**
 * innerJoin works like an SQL inner join, not quite as flexible, 
 *     select a as A, b, frame1.x from frame1, frame2
 *         where frame1.colName1 = frame2.colName2 and {filter} 
 *
 * 
 * @param  {Frame} frame1      [description]
 * @param  {Frame} frame2      [description]
 * @param  {Array<string>} projectList Array of column name mapping example ""
 * @param  {String} joinOn      column name to join on, either "colName" (same name exists o

Cretae file ../src/frame/join-utils/outerJoin.js

import haveFrame from '../haveFrame';
import colName from './colName';
import getTransform from './getTransform';
import localGroupBy from './localGroupBy';
import changeNameTo from './changeNameTo';
import joinOp from './joinOp';
/**
 * [outerJoin description]
 * @param  {[type]} frame1      [description]
 * @param  {[type]} frame2      [description]
 * @param  {[type]} projectList [description]
 * @param  {[type]} joinOn      [description]
 * @param  {[type]} filter      [description]
 * @return {[type]}             [description]
 */
export function outerJoin(frame1, frame2, projectList, joinOn,filter) {
	frame1 = haveFrame(frame1);
	frame2 = haveFrame(frame2);
	//console.log("projectList1:", projectList);
	let {transform, newColNames, mapping} = getTransform(projectList, frame1.columns, frame2.columns);
	let pL = reverseProjectList1(projectList, col1,col2, frame1.columns, frame2.columns);
	//    console.log("projectList3:", pL);
	let


/**
 * array subtraction arr - listToRemove  (return an array with all elements
 * of arr this is not in listToRemove)
 * @param  {Array} arr
 * @param  {Array} listToRemove
 * @return {Array} new array that contains all values of (arr) that are not in listToRemove
 */
export default  function arrRemove(arr, listToRemove) {
	if(!listToRemove || listToRemove.length === 0) return arr || [];
	if(!arr || arr.length === 0) return [];
	return arr.filter( e => listToRemove.indexOf(e) === -1);
}



Cretae file ../src/array/arrayutils/arrSplit.js

/**
 * split an array into n more or less equal length arrays
 * @param  {int} n
 * @param  {Array} arr
 * @return {Array} - return an array of arrays where each inner array is approximately of length n or less
 */
export default  function arrSplit(n,arr) {
	if( n<= 0) throw new Error('array cannot be split to chunks < 0 : \''+n+'\'');
	if(arguments.length === 1) return (arr) => arrSplit(n,arr); // return a function
	let len = arr.length;
	if(n >= le

...

Cretae file ../src/string/strutil/prefixHead.js

import {prefixLen} from './prefixLen';
/**
 * gets the prefix string of 'a' and 'n', for example a = 'asset', b = 'assess', the return will be 'asse'
 * @param  {string} a first string
 * @param  {string} b second
 * @return {string}   the common prefix of both a and b
 */
export default  function prefixHead(a,b) {
	return a.substr(0,prefixLen(a,b));
}


Cretae file ../src/string/strutil/prefixTail.js

import {prefixLen} from './prefixLen';
/**
 * Gets the non-prefix string of 'a' and 'n', for example a = 'asset', b = 'assess', the return will be ['t', 'ss']
 * @param  {String} a [description]
 * @param  {String} b [description]
 * @return {[string]}   the non-common parts of  a and b, the part after the common prefix
 */
export default  function prefixTail(a,b) {
	let pre = prefixLen(a,b);
	return [a,b].map(x => x.substr(pre));
}

Cretae file ../src/string/strutil/prod.js

import {flatten} from '../../array/flatten';
/**
 * This is rthe

Cretae file ../src/dict/utils/SumDict.js

/**
 *  Dictionary to count the number of key value pairs,
 *  Note for any key we only support one value, if 
 */
export class SumDict {
	constructor(klass,dflt) {
		this.klass = klass || Accum;
		this._dict = {};
		this._dflt = dflt;
	}

	set(key,value) {
		let acc = this._dict[key];
		if( !acc ) {
			acc = new this.klass();
			this._dict[key] = acc;
		}
		acc.add(value===undefined?this._dflt:value);
		return this;
	}

	add(key,value) {return this.set(key,value); }
	put(key,value) {return this.set(key,value); }
	get(key) {return this._dict[key]; }

	asZip() {
		let keys = Object.keys(this._dict);
		return flatten(
			keys.map(k => {
				let acc = this._dict[k];
				if( !acc ) return [[k, undefined]];
				// O: KEEP
				return acc.map((val) =>[k, val]);
			} ) ,1);
	}

	get dict() { return this._dict; }
}
/**
 * create a dict from zip array - an array of key value pairs
 * @param  {[[key,value]]} aZipList [description]
 * @return {SumDict}    

Cretae file ../src/utils/objutils/fromKeyValueList.js

/**
 * [fromKeyValueList description]
 * @param  {[type]} list [description]
 * @return {[type]}      [description]
 */
export default function fromKeyValueList(list) {
	return list.reduce( (tab,[k,v]) => setKey(tab, k, v),{});
}


Cretae file ../src/utils/objutils/setKeyV.js

/**
 * [setKeyV description]
 * @param {[type]} obj [description]
 * @param {[type]} key [description]
 * @param {[type]} v   [description]
 */
export default function setKeyV(obj,key,v) {
	if(arguments.length == 1) 
		return obj[key];
	else if( v === undefined) return obj;
	else {
		let res = obj[key];
		if( res ) res.push(v);
		else obj[key] = [v];
		return obj;
	}
}


Cretae file ../src/utils/objutils/toKeyValueListV.js

/**
 * [toKeyValueListV description]
 * @param  {[type]} obj [description]
 * @return {[type]}     [description]
 */
export default function toKeyValueListV(obj) {
	let expand = Object.keys(obj).map(doExpand);
	return flatten(expand,1);
	f

fs.mkdirSync(../src/utils/sort-helper);
Cretae file ../src/utils/sort-helper/toNumber.js


export function toNumber(str) {
	if(!str) return undefined;
	let len = str.length,negate = 1;
	let res = +str;
	if(!isNaN(res)) return res;
	if( res === Infinity ) return undefined;
	res = 0;
	if( str.length > 1 && (str[0] === '$'|| (str[0] ==='-' && str[1] === '$')) ) {
		let i = 1;
		if(str[1] === '-' || str[0] === '-'){
			negate = -1;
			i++;
		} 
		if( i === len ) return undefined;
		while(i<len) {
			let c = str[i];
            
			if( c === ',') { i++; continue; }
			if( c === '.') {
				let fraction = Number(str.substr(i));
				if( !isNaN(fraction)) {
					return 1.0*negate*res+fraction;
				}
				return undefined;
			}
			if(c < '0' || c > '9') return undefined;
			res = res*10+(+c);
			i++;
		}
		return res;
	} else {
		let i = 0;
		if(str[0] === -1){
			negate = -1;
			i++;
		} 
		if( i === len ) return undefined;
		while(i<len) {
			let c = str[i];
			if( c === ',') { i++; continue; }

undefined

In [9]:
function process(_jsFile) {
    
}
fs.existsSync("../src/array/_arrayutils.js")

true

In [98]:
transform("../src/frame/frame-utils","import a, {aaa,bbb,cc} from \"../array/arrEQ\"  ");


'../../array/arrEQ'

In [99]:
transform("../src/frame/frame-utils","import a, {aaa,bbb,cc} from \"./frame\"  ");

'../frame'

undefined